In [271]:
import pandas as pd
import json
import os
import glob

In [18]:
df = pd.read_csv('2020-04-10/metadata.csv', low_memory= False)

In [19]:
df.shape

(47298, 18)

In [20]:
df = df[df.has_pdf_parse == True]

In [23]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
3,zp9k1k3z,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(73)90176-9
5,wwf90zxt,212e990b378e8d267042753d5f9d4a64ea5e9869,Elsevier,Infectious diarrhea: Pathogenesis and risk fac...,10.1016/0002-9343(85)90367-5,NaN,2861742.0,els-covid,Abstract Our understanding of the pathogenesis...,1985-06-28,"Cantey, J.Robert",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(85)90367-5
6,dlh93ax6,bf5d344243153d58be692ceb26f52c08e2bd2d2f,Elsevier,New perspectives on the pathogenesis of rheuma...,10.1016/0002-9343(88)90356-7,NaN,3052052.0,els-covid,Abstract In the pathogenesis of rheumatoid art...,1988-10-14,"Zvaifler, Nathan J.",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(88)90356-7
7,i94lyfsh,ddd2ecf42ec86ad66072962081e1ce4594431f9c,Elsevier,Management of acute and chronic respiratory tr...,10.1016/0002-9343(88)90456-1,NaN,3048091.0,els-covid,"Abstract Pharyngitis, bronchitis, and pneumoni...",1988-09-16,"Ellner, Jerrold J.",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(88)90456-1
8,vs5yondw,a55cb4e724091ced46b5e55b982a14525eea1c7e,Elsevier,Acute bronchitis: Results of U.S. and European...,10.1016/0002-9343(92)90608-e,NaN,1621745.0,els-covid,"Abstract Acute bronchitis, an illness frequent...",1992-06-22,"Dere, Willard H.",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0002-9343(92)90608-e


In [46]:
with open('C:/Users/eovit/Python/Kaggle/2020-04-10/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/01d162d7fae6aaba8e6e60e563ef4c2fca7b0e18.json', 'rb') as f:
    dat  = json.load(f)

In [358]:
def json_conv(json_obj):
    """
    This function handles the native JSON schema from the COVID pdf-conversions\
    
    Input - > Json object
    
    Output - > df of title, author, abstract, full text
        i.e it is a list of length 4
        
    This function does not catch all exceptions, like colons, admission of the keyword being looked at

        
    """
    information = []
    title = ''
    all_auths = []
    all_body = []
    abst = ''

    if json_obj['metadata']:
            
        if json_obj['metadata']['title']:
            title += str(json_obj['metadata']['title'])
            
        #handling naming conventions in the json 
        for i in json_obj['metadata']['authors']:
            prop_name = ''
            if i['middle']:
                prop_name += str(i['first']) + ' ' +  str(i['middle']) + ' ' + str(i['last'])
            else:
                prop_name += str(i['first']) +  ' ' + str(i['last'])
            all_auths.append(prop_name)
            
        if json_obj['abstract']:
            for j in json_obj['abstract']:
                if j['text']:
                    abst += str(j['text'])
                    
        if json_obj['body_text']:
            body_txt = ''
            for l in json_obj['body_text']:
                if l['text']:
                    body_txt += str(l['text'])
            all_body.append(body_txt)
            
            
    information.append(title)
    information.append(all_auths)
    information.append(abst)
    information.append(body_txt)
    return information

In [359]:
def json_opener(path):
    #wildcard to open all jsons
    all_data = []
    col_names = ['title', 'authors', 'abstract', 'all_body']
    for filename in glob.glob(os.path.join(path, '*.json')):
        with open(filename, 'rb') as f:
            data = json.load(f)
            d_con = json_conv(data)
        all_data.append(d_con)
    df = pd.DataFrame(all_data, columns = col_names)
    return df

In [360]:
if __name__ == '__main__':
    path = 'C:/Users/eovit/Python/Kaggle/2020-04-10/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/'
    json_extract = json_opener(path)
    df = pd.DataFrame(json_extract)
    df.to_csv('C:/Users/eovit/Python/Kaggle/research_papers_v1.csv')